# Fiber Photometry Demo

This notebook contains exercises split across multiple lessons that span the following primary topics
* strings
* conditionals
* lists
* pandas
* dictionaries
* visualization

The exercise modules are centered around analysis of neuroscience fiber photometry data, which involves recording __bulk__ changes in brightness of activity-dependent fluorescent proteins that are expressed in cells of interest. Typically the activity data stream consists of a single vector time-series (i.e. fluorescence intensity measurements acquired across time). For this particular dataset (acquired by Dr. Adam Gordon-Fennell), animals were periodically allowed to consume liquid rewards in "access periods." Both the onsets of the access periods and animal licks were recorded. The data consist of the following files:

* *_events.csv : table where rows contain the event type (access period or lick) and time of occurrence (in seconds)
* *_streams_session.csv : table where rows are samples acquired across time for the photometry recording. For each sample, the table details the channel, time, and fluorescence value. __The data contain activity traces for a 405nm isosbestic control channel and a 465nm GCaMP signal.__

The exercises follow a sequence starting from loading in (behavioral and neural stream) data from tabular files (excel CSVs), examining the data and extracting relevant data streams, preprocessing the data, and visualizing the activity. In the 2nd half of the exercises, we incorporate the behavioral data and generate activity plots centered around the behavioral events. 


## Intro exercise: Importing packages

Usually the very beginning of a script (besides commentary and markdown) consists of package imports. Below I've already specified a few packages to import. We still need to import the `pandas` package. Go ahead and add pandas to the import list and abbreviate it to `pd`

If the import is successful, the following lines should execute

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True) # turns scientific notation off

In [ ]:
pd.__version__

In [ ]:
# here I set the directory and file name that we want to load
fdir = r'sample_data/' 
fname = '2022_06_10_abb12'

In [ ]:
pd.read_feather(fdir + fname + '_events.feather') # just a temporary view of the data

## Conditionals exercise 1: Detecting and loading data in different file formats.

The cell below defines the absolute file path (`epoc_data_path`) for the behavioral data. Currently we are loading in the .feather file, which is just another file format that can store tabular data, as opposed to csv or xlsx formats.

In [ ]:
# note os.path.join is a function that simply merges and formats string arguments together such that they are compatible with other file loading functions
epoc_data_path = os.path.join(fdir, f'{fname}_events.feather') 
epoc_data_path

First let's start with a practice example:

Notice that we are using the pandas package (abbreviated in the import code section at the top of this notebook as `pd`) to load the tabular data. We will learn more about it later, but for now, pandas is a package that adds tabular data processing capabilities to python.

In [ ]:
if 'feather' in epoc_data_path:
    print('loading feather file')
    epoc_data = pd.read_feather(epoc_data_path)

### _Complete the following_ (Conditionals exercise 1)

I've provided the contents of the if statements, please replace the lines marked with my comments with an `if` statements checking for `'feather'` files, an `elif` checking for `'csv'` files, and a catch-all `else` statement for the variable `epoc_data_path`.  

In [ ]:
if 'feather' in epoc_data_path:
    print('loading feather file')
    epoc_data = pd.read_feather(epoc_data_path)
elif 'csv' in epoc_data_path:
    print('loading csv file')
    epoc_data = pd.read_csv(epoc_data_path)
else:
    print('Not a valid file type')

In the sample_data folder, we also have a csv version - try changing the behavioral data path above (`epoc_data_path`) to the csv version, then rerun the above.

Pandas and methods preview! We'll use these to view the data that we loaded.

In [ ]:
# Calling the .head() method on our pandas dataframe will show the top 5 entries.
epoc_data.head()

## Conditionals exercise 2: Checking contents of lists using conditionals and boolean operators
### Also list preview! Lists (name is kind of self-explanatory) are a type of data format that stores a sequence of items

The preview of the behavioral data table above exercise doesn't really tell us how many behavioral conditions there are (event_id_char column) since there are so many row entries. We can have python look through all entries in the event_id_char column and tell us what unique entries there are by using the .unique() method. We will go over what coding concept a method is in later lessons, but for now, just know it will return all unique entries for a given pandas dataframe/series.

In [ ]:
event_conditions = list(epoc_data['event_id_char'].unique()) # grab unique condition names
event_conditions

### _Complete the following_ (Conditionals exercise 2)

Now that we have a list that contains all unique conditions present in the data, we might want to automatically check if particular conditions are present. To do that we could leverage if statements and boolean operators. Below, please fill in the underscored parts to enable 
1) the first portion of the if statement to check if BOTH 'access_period' AND 'lick' are in the conditions list (`event_conditions`).
2) the second portion of the statement (`elif`) to check for at least one of the above conditions are present
3) bonus is to add a catch-all in case neither condition is present.

In [ ]:
if 'access_period' in event_conditions and 'lick' in event_conditions:
    print('Both conditions present')
elif 'access_period' in event_conditions or 'lick' in event_conditions:
    print('One out of the two conditions present')

## Pandas exercise 1: Exploring contents of a dataframe

Use and familiarize yourself with various pandas dataframe methods to get a sense of the contents in the behaviora epoch dataframe (`epoc_data`).

### _Complete the following_ (Pandas exercise 1)

__Please use the nunique, info, head, tail, shape methods (in separate cells). Read the documentation to understand what info is output and what arguments you can input into the method to customize the information delivered. In another cell, please also use dataframe slicing to identify which row do the event conditions switch (from access_period to lick, for example - you will have to determine this empirically or check the csv file!).__

In [ ]:
epoc_data.nunique() # provides count of unique entries in each column
epoc_data.info() # can identify if there is missing data

In [ ]:
epoc_data.head() # check the first 5 entries 

In [ ]:
epoc_data.tail() # check the last 5 entries

In [ ]:
epoc_data.iloc[95:105] # use dataframe slicing/indexing to find which line the behavioral conditions change from access_period to lick

In [ ]:
epoc_data.shape # check the dimensions of the dataframe
# we can also get a sense of how many events (y dimension) there are. By using this method, the first dimension outputted is y and the 2nd in x.

## Pandas exercise 2: Obtaining data from a dataframe column based on information in another column.

We are working with neural activity data that was synchronized to events that happened during the recording. Our behavioral dataframe contains event times (event_ts column) for two different types of behavioral events (`event_id_char` column: access_periods, licks, and high-fives). Here let's figure out how to extract event times for a specific condition, `access_period`.

### _Complete the following_ (Pandas exercise 2.1)

First we need to identify which rows in `epoc_data` correspond to the `access_period` condition. __Please use the `==` comparison to make this comparison and assign this boolean list/series to the variable below__

In [ ]:
boolean_access_period = epoc_data['event_id_char'] == 'access_period'
boolean_access_period

Now let's use our boolean mask that indicates which rows are `access_period` trials to extract the corresponding time stamps of those events. 

### _Complete the following_ (Pandas exercise 2.2)

__Please fill in the line below with dataframe conditional selection to extract timestamps from the `event_ts` column and `access_period` trials indexed using the `boolean_access_period` variable.__

In [ ]:
# grab event times for condition 1
event_one = epoc_data['event_ts'][boolean_access_period].values
event_one

# Now we'll begin to work with the time-series data

## Pandas, math, and functions combined exercise: Correcting photometry traces with isosbestic channel

Our goal here is to ultimately plot the photometry activity trace across the whole session in one window. Specifically we want to plot the 405 isosbestic, 465 GCaMP, and the 405-corrected signal in the same plot. 

In the cell immediately below, we'll read in the raw photometry activity table data. It has columns for timestamps, raw activity values, and channel (405nm vs 465nm).

In [ ]:
data_path = os.path.join(fdir, f'{fname}_streams_session.feather')

data = pd.read_feather(data_path)
data

We need to extract the raw activity signals for the 405nm and 465nm channels separately (they are currently stacked on top of eachother in the dataframe). 

### _Complete the following_ (Pandas exercise 3.1)

__Please use conditional selection again (similar principle as the previous exercise) to first identify rows that correspond to 405nm and then use that boolean mask to extract activity data (`signal`) from the `data` dataframe.__ 

*** hint: the 405 and 465 entries are strings, not actual numbers, so you will need to encompass the numbers with quotes like `'405'`

In [ ]:
"""
I've kept the .values method because it converts the data 
from a pandas series to a numpy array, which in my opinion 
and experience is easier to work with.
"""
data_405 = data['signal'][data['channel'] == '405'].values
data_465 = data['signal'][data['channel'] == '465'].values

Next let's obtain the time-stamps across all the datapoints. Remember this information is also in the `data` variable (column is called `'time'`). In this particular "tidy" format of the `data` sheet, we have 405 channel data stacked on top of the 465 data. Because of that, each channel possesses a duplicate of the time-stamps across the whole session (you can see in the cell below that when the channel transitions to 465nm, the time stamps reset). So we only need the time-stamps from one channel.

In [ ]:
data.loc[31920:31928, 'time':'channel']

__Below, please extract the `'time'` values from the `data` dataframe by filling in the underscored section below, and assigning to the variable `tvec` (time vector, abbreviated).__

### _Complete the following_ (Pandas exercise 3.2)

`tvec_zeroed` is simply tvec but shifted such that the first timepoint is 0. Note we are only using this for plotting purposes for this particular exercise section. We do not need to normalize the tvec for subsequent exercises!

In [ ]:
tvec = data['time'][data['channel'] == '405'].values 
tvec_zeroed = tvec - data['time'][0] # for plotting trace starting from time 0

print(tvec_zeroed)

A popular method for correcting the GCaMP signal using the isosbestic signal is to first calculate a linear least squares (LLS) fit between the GCaMP and isosbestic signal, apply the transform to the isosbestic signal (fitted control), and subtract the fitted control trace from the GCaMP signal

I've precoded a couple of functions that will help with this process. 

The first function will perform the fit between the 405 and 465 channels, and apply the transform to the 405 isosbestic channel. This generates a fitted control as the output.

What's going on mathematically here? The `np.polyfit(x,y,1)` function is essentially a linear regression `y=mx+b` equation. Let's think of it in this context.

<img src="pics\LLS.jpg" width="420" />

The second function will apply a delta F/F normalization calculation. This consists of subtracting the fitted control signal from the 465 GCaMP signal sample-by-sample. This yields the fully-corrected signal that can be used for downstream analysis.

In [ ]:
def controlFit(control, signal):
    
    # GuPPY version
    # function to fit control channel to signal channel

    p = np.polyfit(control, signal, 1) # p are the polynomial coefficients for the line that best fits control (x) to signal (y)
    arr = (p[0]*control)+p[1] # apply transform coefficients to control channel to generate fitted control
    return arr


def deltaFF(signal, control):
    
    # function to compute deltaF/F using fitted control channel and filtered signal channel

    res = np.subtract(signal, control) # numerator of F(t)-f0
    normData = np.divide(res, control) # (F(t)-f0)/F0
    normData = normData*100

    return normData

fitted_control = controlFit(data_405, data_465) # this function would be helpful if we wanted to correct signals from multiple fluorophore channels
corrected_data = deltaFF(data_465, fitted_control) # this function would be helpful everytime we needed to compute dF/F

Here's the final plot! No need to do anything besides execute the cell below and enjoy the fruits of your labor!

Notice how this normalization method corrects the photobleaching/slow drift trend and also sets the baseline close to 0.

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(tvec_zeroed, data_405)
plt.plot(tvec_zeroed, data_465)
plt.plot(tvec_zeroed, fitted_control)
plt.plot(tvec_zeroed, corrected_data)
plt.xlabel('Time (s)')
plt.ylabel('Fluorescence (dF/F)')
#plt.xlim([170, 400]) # we can zoom in to see how motion artifacts can be corrected for
plt.legend(['405', '470', 'fit curve', 'lls-corrected']);

### Dictionary Exercise 1: Dividing event times up into their respective conditions. These event times can then be organized into a dictionary for storage purposes.

Now that we understand how the behavioral and time-series data are formatted, let's start to think about how we can link the two data formats. First we know there are multiple event types (e.g. ccess_period and licks) in the behavior and that for each event we have a corresponding time point that it happened at. When we ultimately view the event-related data, we probably want it organized by event type. So the next step is to group the event times based on event type, which can be done using.

### _Complete the following_ (Dictionary exercise 1)

__Please fill in the cell below to extract time-stamps for each event and organize them by conditions into a dictionary.__

** hint the lines within the for loop that extracts the event times can be formatted similar to how we extracted time-stamps for the 'access_period' events back in the pandas lesson. The variable name was `event_one` and we used conditional selection. Think about how to interface a for loop with conditional selection. 

Also, just a reminder to use the .values method on the selected series to extract data as a numpy array, if you prefer.

In [ ]:
# grab event times for all conditions via for loop
event_times_dict = {}
for event_name in epoc_data['event_id_char'].unique():
    
    boolean_event_type = epoc_data['event_id_char'] == event_name
    event_times_dict[event_name] = epoc_data['event_ts'][boolean_event_type].values

event_times_dict

### Just reading: Convert behavioral event times to samples: to prepare for indexing and snipping out trial activity windows.

#### Requires knowledge on: dictionaries and functions 

The whole-session activity trace, while nice to see, gives us very limited understanding of what is happening when the animal performs a behavior or reacts to a manipulation. 
* To get a better sense, we will need to get closeup snapshots of activity centered around events. 
* In other words, we need to identify when in the activity data an event occurred and then extract the surrounding datapoints.

The activity data itself, in isolation, is not characterized by units of time, rather each item in the vector is a "sample" in the recording and does not hold any information on timing. We do however have the time vector column (we assigned this to the `tvec` variable) that indicates for each frame what time they were acquired at, which will be useful.

In [ ]:
data['signal']

In [ ]:
data.head()

On the other hand our behavioral events are in units of time, and to make things slightly complicated, they may not be occurring exactly when a photometry data point was acquired. Also in our case, there are missing timestamps for our photometry data, so directly converting behav time stamps to photometry data samples does not work.

So we need to identify the photometry datapoint that occurred nearest to when the behavioral event happened. To do this, we use the time vector `tvec` and each behavioral event to match the closest datapoint to an event occurrence

We can use this line of code : `np.argmin(abs(tvec - event_time))`

Here by subtracting a single event time from tvec, the closest datapoint timestamp to the event time will be 0 or near-zero. This sets things up nicely for us to use `np.argmin()` (we learn about this in the numpy lesson) to identify the index of the minimum value in the vector. `abs` is used to flip the sign of negative numbers so that they don't count in the minimum value search.

In [ ]:
print(f'First event from condition one (event time): {event_one[0]} seconds')

print(f"The photometry sample that corresponds to {event_one[0]} sec is {np.argmin(abs(tvec - event_one[0]))}")

print(f"Photom sample {np.argmin(abs(tvec - event_one[0]))}'s time: {tvec[4224]} seconds") # if dataset changes, replace number in tvec[4224] with calculated sample number

Notice the slight difference in the actual event time compared to the calculated photometry sample time because the two data aren't synchronously acquired (at least in this dataset).

In the cell below, I've turned the above time-stamp to samples (using the `tvec` variable) conversion into a repeatable function for our next exercise.

In [ ]:
# turn the above code into a function
def get_tvec_sample(tvec, time):
    return np.argmin(abs(tvec - time))

### Dictionary exercise 2: To analyze all trials, let's do some organization of the event conditions and trial times into a dictionary

In this exercise let's create a dictionary similar to the one in exercise 1, but now all the contents will be in samples instead of time. This will make it easier for us to extract data from the whole session activity trace using the appropriate indexing. 

### _Complete the following_ (Dictionary exercise 2)

__Although there are different ways to do this, the method that I would like you to practice is using a for loop to go through the event types. Within this for loop, we will use a second for loop to go through the events in `event_times_dict` and convert each time-stamp to samples using the function I provided above. IMPORTANTLY, this method requires the use of initializing an empty list that will be populated with the converted event samples and finally assigned to a dictionary key. Please fill in the lines below.__

In [ ]:
event_samples_dict = {} # initialize your dictionary
for condition_name in epoc_data['event_id_char'].unique(): # loop through conditions
    
    tmp_list = [] # initialize your list
    
    # go through each event and compute the sample/frame that it occurred on (b/c the list is currently in seconds)
    for event in event_times_dict[condition_name]: # loop through events
        tmp_list.append(get_tvec_sample(tvec, event))
        
    # once we go through converting each time to sample, and add to a list iteratively,
    # store that list to its corresponding condition in the dictionary
    event_samples_dict[condition_name] = tmp_list

The two cells below allow us to see if the exercise code succeeded: 1) keys should show us all the event conditions names, 2) calling the value contents of a particular key of the dictionary should show us a list of event samples.

In [ ]:
event_samples_dict.keys()

In [ ]:
event_samples_dict['access_period']

### Numpy (and dictionary) exercise 1: Let's analyze data from one trial

To extract trial-related activity data from the whole-session recording, we need the following information: 
* the event onset in units of samples (since the activity vector inherently does not hold information about time)
* start and end times for the window
* the sampling rate to convert start/end times to relative sample indices

### _Complete the following_ (Numpy exercise 1.1)

__Calculate the sampling rate using the time-vector that was calculated in the pandas exercise (`tvec`, which indicates the timestamp, in seconds, of each sample):__
1. calculate the difference between subsequent value pairs of `tvec`
2. take the mean across all entries of step 1
3. divide 1.0 by the result from step 2, and assign to the variable `fs`

In [ ]:
fs = 1.0/np.mean(np.diff(tvec))
print("sampling rate:", fs)

Let's first define the start and end times for our template event window (`event_window_time`). Then multiply those times by the sampling rate to obtain the start/end sample indices `event_window_samples`.

### _Complete the following_ (Numpy exercise 1.2)

Define a 1D vector for the variable `event_window_time` that contains 2 elements: 1) the event window start time and 2) the event window end time. I recommend using -1 and 10 so that things line up with the markdown text below; however you can adjust to your liking.

In [ ]:
# initialize start/end times for trial window
event_window_time = np.array([-1, 10])

In [ ]:
event_window_samples = (event_window_time*fs).astype(int) # samples are integers
event_window_samples

If you look at the indices above, -20 corresponds to -1 seconds multiplied by the sampling rate (`fs`=20, which we calculated earlier in this notebook) and 199 corresponds to 10 seconds * `fs` (side note: it's not 200 b/c we used `np.arange`, which cuts off the last number).

With the information above, we need to compute a vector `trial_template_indices` that contains the sample indices relative to event onset (sample 0) in preparation for extracting event-related activity in a trial window. Specifically we wanted to extract a window of time from -1 to 10 seconds relative to event onset (encoded in the variable `event_window_time` - see cell above)

In [ ]:
# generate vector of every sample between start and end samples
# then offset to specific event's window by adding the event sample
trial_template_indices = np.arange(event_window_samples[0], event_window_samples[1])
trial_template_indices

Adding the event sample to every entry of the `trial_template_indices` gives us the samples required to extract activity for ___this specific trial___ in the -1 to 10 second time window around event onset!

In [ ]:
# we demonstrated how to convert time to samples in the previous lesson's exercise above  
first_event_sample = np.argmin(abs(tvec - event_one[0]))
first_event_sample

In [ ]:
trial_samples = trial_template_indices + first_event_sample
trial_samples

Below, we generate a vector that maps time onto samples in the event trial. This is for plotting visualization purposes.

In [ ]:
trial_window_num_samples = event_window_samples[1] - event_window_samples[0]
tvec_trial = np.linspace(event_window_time[0], event_window_time[1], trial_window_num_samples)
tvec_trial

### _Complete the following_ (Numpy exercise 1.3)

Given our vector `trial_samples` that represents the indices in our recording that corresponds to the specific trial we want to visualize, use those indices to slice into the whole session's activity trace (`corrected_data`) in preparation for plotting. Assign it to the variable `corrected_data_trial`.

It's very simple! You only need the variables outlined above.

In [ ]:
corrected_data_trial = corrected_data[trial_samples]

In [ ]:
plt.plot(tvec_trial, corrected_data_trial)

In [ ]:
# CZ ADDED 4/3/2025

#trial 1
event_sample = np.argmin(abs(tvec - event_one[0])) # reminder: this fill find the frame that the event occurs during
indices_to_extract = trial_template_indices + event_sample
plt.plot(corrected_data[indices_to_extract])

#trial 2
event_sample = np.argmin(abs(tvec - event_one[1])) 
indices_to_extract = trial_template_indices + event_sample
plt.plot(corrected_data[indices_to_extract])

#trial 3
event_sample = np.argmin(abs(tvec - event_one[2])) 
indices_to_extract = trial_template_indices + event_sample
plt.plot(corrected_data[indices_to_extract])

### Numpy (and dictionary) exercise 2: Let's analyze data from ALL trials

To extract data from each trial across the session, we will create a loop that iterates through each event and adds the event onset sample to each entry in the `trial_template_indices` vector, like we did in the single trial activity extraction in the dictionary exercises. Fortunately, in those exercises, we generated a dictionary  (`event_samples_dict`) that contains the onset samples for each event occurrence, making it easier for us to loop through all events to calculate the samples to extract.

To be able to save the trial snippits for each conditions, our code block will be a bit more sophisticated than we are used to and employ a couple of nested for loops. The first one will loop through trial conditions broadly and the block within will go through each trial for a given condition and generate an array containing trial snippits. This array will be assigned to a dictionary key.

### _Complete the following_ (Numpy exercise 2.1)

Do achieve the above, use the partially filled out code below and complete the following:

1. Initialize an array that can be populated with activity trace snippits from each trial (called `tmp_trial_array`)
2. Within the event for loop, assign activity trace snippits for each trial to the corresponding row in `tmp_trial_array`
3. Assign the completed `tmp_trial_array` to a `data_trial_dict` key.

In [ ]:
data_trial_dict = {}
for condition_name in epoc_data['event_id_char'].unique(): # loop through conditions
    
    # initialize a numpy array to populate with trial-extracted data
    tmp_trial_array = np.ones([len(event_samples_dict[condition_name]), trial_window_num_samples])*np.nan
    
    # use trial index template, offset with event sample, and extract trial data
    for idx, event_sample in enumerate(event_samples_dict[condition_name]): # loop through events
        tmp_trial_array[idx,:] = corrected_data[trial_template_indices + event_sample]
    
    # once data are fully populated, add to dictionary
    data_trial_dict[condition_name] = tmp_trial_array

In [ ]:
data_trial_dict['access_period'].shape

# we expect the dimensions to be 100 trials by 220 samples

### _Complete the following_ (Numpy exercise 2.2)

The final quick exercise is to take the array above that represents activity snippits from all "access_period" trials (`data_trial_dict['access_period']`) and take the mean across trials. Assign this output to the variable `mean_access_period` and you will get a sneakpeak of the visualization lesson/exercises.

In [ ]:
mean_access_period = np.mean(data_trial_dict['access_period'], axis=0)

plt.plot(mean_access_period)

What if we suspect there is a trial that contains an artifact in a certain time window? How do we efficiently exclude it from the analysis? Let's use NaN (not a number) entries to replace unwanted segments of values. 

### _Complete the following_ (Numpy exercise 2.3) - Excluding numpy numerical data

For this let's focus on a single trial. I've predefined the window of time in our first `access_peiod` event epoch to exclude (`exclude_times`). Using the start and end samples (`exclude_start_samp`, `exclude_end_samp`), slice into `corrected_data` and replace the appropriate window with NaNs. Then plot that trial's activity trace.

* __Break glass in case of emergency: if you mess up and `corrected_data` gets drastically altered, you will have to re-execute the code below. This generates the `corrected_data` variable - which was originally calculated in the pandas exercise 3.2__

`corrected_data = deltaFF(data_465, fitted_control)`

In [ ]:
corrected_data = deltaFF(data_465, fitted_control)

In [ ]:
exclude_times = [0, 5] # time window to exclude/convert to nans

exclude_start_samp = (event_samples_dict['access_period'][0] + exclude_times[0]*fs).astype(int)
exclude_end_samp = (event_samples_dict['access_period'][0] + exclude_times[1]*fs).astype(int)

In [ ]:
corrected_data[exclude_start_samp:exclude_end_samp] = np.nan * np.zeros( exclude_end_samp-exclude_start_samp)

In [ ]:
print(corrected_data[trial_samples[0]:trial_samples[-1]])
plt.plot(corrected_data[trial_samples[0]:trial_samples[-1]])

### _Complete the following_ (Visualization exercise 1): Lineplots

Given the timestamps vector for a trial window (`tvec_trial`) and the dictionary above that contains trial snippits of fluorescence activity for each behavioral condition (`data_trial_dict`), generate a single line plot that shows two traces/lines: the mean (across trials) for both behavioral conditions.

Please also add axis labels and a legend.

For bonus points, can you identify which lines of code contain redundant segments and thus would be ideal for turning into a for loop? If so, try to implement such a for loop.

Hint: use the plot function in matplotlib. 

In [ ]:
cond_names = list(data_trial_dict.keys())

plt.plot(tvec_trial, np.mean(data_trial_dict[cond_names[0]], axis=0))
plt.plot(tvec_trial, np.mean(data_trial_dict[cond_names[1]], axis=0))
plt.ylabel('Fluorescence', fontsize=14)
plt.xlabel('Time', fontsize=14)
plt.legend(cond_names, fontsize=14)



### _Complete the following_ (Visualization exercise 2): Trial-resolved heatmaps

Now let's focus on one condition's activity for ALL trials - it may be useful to examine how heterogeneous or homogeneous activity is across trials. Here, pull data out from one condition and generate a heatmap. Please include axis labels and a colorbar.

If you are feeling spicy, try creating a figure that contains two subplots so you can visualize both condition's trial-resolved activity.

In [ ]:
plt.imshow(data_trial_dict[cond_names[0]], aspect='auto')
plt.ylabel('Trial', fontsize=14)
plt.xlabel('Sample', fontsize=14)
cbar = plt.colorbar()
cbar.set_label('Fluorescence', fontsize=14)